Здесь используется датасет данных о результатах тестирования крови пациентов с инфекционными заболеваниями различной тяжести.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, log_loss
import scipy.stats as ss
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
df = pd.read_csv('clean_data.csv', encoding = 'iso-8859-1')
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Patient ID,Patient age quantile,Patient gender,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,0,4393,4393,f3f71ca05b07798,11,1.0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5583,5583,73811966f4b1141,19,1.0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5615,5615,2733fac0d3f7138,15,0.0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5221,5221,c88564b2564a8c1,0,0.0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2573,2573,0d3d7beb19b3afc,19,1.0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
def read_data():
    df = pd.read_csv('clean_data.csv')
    df = df.sample(frac=1).reset_index(drop=True)
    df = df.replace('not_detected', 0)
    df = df.replace('detected', 1)
    df = df.replace('positive', 1)
    df = df.replace('negative', 0)
    df = df.replace('ìàð.65', 0)
    df = df.replace('nan', np.nan)

    
    df = df.dropna(subset=['Platelets.1','Hb saturation (venous blood gas analysis)'])
    df = df.sample(frac=1).reset_index(drop=True)
    df_1 = df[df['Patient addmited to regular ward (1=yes, 0=no)'] == 1]
    df_2 = df[df['Patient addmited to semi-intensive unit (1=yes, 0=no)'] == 1]
    df_3 = df[df['Patient addmited to intensive care unit (1=yes, 0=no)'] == 1]
    df_ = pd.concat([df_1, df_2, df_3])
    df = df_
    df = df.sample(frac=1).reset_index(drop=True)
    return df


In [8]:
class LogisticRegression():
    def __init__(self, fit_intercept = True, lr=1e-3, num_steps=1000, tol=1e-5):
        self.X = None
        self.Y = None
        self.beta = None
        self.num_steps = num_steps
        self.lr = lr
        self.tol = tol
        self.fit_intercept = fit_intercept

    def fit(self, X, Y):
        if X is None:
            intercept = np.ones((Y.shape[0], 1))
            self.X = np.hstack((intercept)).reshape(-1, 1)
        else:
            if self.fit_intercept:
                intercept = np.ones((X.shape[0], 1))
                self.X = np.hstack((intercept, X.copy()))
            else:
                intercept = np.zeros((X.shape[0], 1))
                self.X = np.hstack((X.copy(),))
        self.Y = Y.copy()
        self.beta = np.ones(self.X.shape[1])
        #if not self.fit_intercept:
        #    self.beta[0] = 0
        self.logistic_regression(self.X, self.Y)

    def sigmoid(self, X):
        z = X @ self.beta
        return 1 / (1 + np.exp(-z))

    def loss(self):
        h = self.sigmoid(self.X)
        cost = (((-self.Y).T @ np.log(h))-((1-self.Y).T @ np.log(1-h))).mean()
        return cost

    def logistic_regression(self, X, Y):
        loss_history = [0]
        for step in range(self.num_steps):
            predictions = self.sigmoid(self.X)
            self.beta += self.lr * np.dot(self.X.T, self.Y - predictions)
            loss_step = self.loss()
            if abs(loss_step - loss_history[step]) < self.tol:
                break
            loss_history.append(loss_step)
        return self.beta

    def predict(self, X):
        if isinstance(X, int):
            intercept = np.ones((X, 1))
            X_ = np.hstack((intercept)).reshape(-1, 1)
        else:
            if self.fit_intercept:
                X_ = X.copy()
                intercept = np.ones((X_.shape[0], 1))
                X_ = np.hstack((intercept, X_))
            else:
                X_ = X.copy()
                intercept = np.zeros((X_.shape[0], 1))
                X_ = np.hstack((X_,))

        return np.round(self.sigmoid(X_))

    def predict_proba(self, X):
        if isinstance(X, int):
            intercept = np.ones((X, 1))
            X_ = np.hstack((intercept)).reshape(-1, 1)
        else:
            if self.fit_intercept:
                X_ = X.copy()
                intercept = np.ones((X_.shape[0], 1))
                X_ = np.hstack((intercept, X_))
            else:
                X_ = X.copy()
                intercept = np.zeros((X_.shape[0], 1))
                X_ = np.hstack((X_,))

        return self.sigmoid(X_)


Названия хронических заболеваний и симптомов соответственно в списках diseases_name и symptoms_name. 

In [16]:
diseases_name = [ 'Patient age quantile', 
       'Patient gender', 
       'Hematocrit', 'Hemoglobin', 'Platelets.1',
       'Red blood Cells',
       'Lymphocytes', 'Leukocytes', 'Basophils',
       'Eosinophils #', 
       'Basophils #', 'Monocytes #',
       'Red blood cell distribution width (RDW)','pCO2 (venous blood gas analysis)',
       'Hb saturation (venous blood gas analysis)',
       'Base excess (venous blood gas analysis)',
       'pO2 (venous blood gas analysis)',
       'Total CO2 (venous blood gas analysis)',
       'pH (venous blood gas analysis)',
       'HCO3 (venous blood gas analysis)',  'Neutrophils', 'Urea', 'Proteina C reativa mg/dL',
       'Creatinine', 'Potassium', 'Sodium']

symptoms_name = ['Patient addmited to regular ward (1=yes, 0=no)', 'Patient addmited to semi-intensive unit (1=yes, 0=no)',
                 'Patient addmited to intensive care unit (1=yes, 0=no)']
fraction = 0.80
print('Сейчас датасет содержит данные о {} пациентах и обучается на данных о {} пациентах.'.format(df.shape[0],
                                                                                                   int(fraction*df.shape[0])))
df_1 = df[df['Patient addmited to regular ward (1=yes, 0=no)'] == 1]
df_2 = df[df['Patient addmited to semi-intensive unit (1=yes, 0=no)'] == 1]
df_3 = df[df['Patient addmited to intensive care unit (1=yes, 0=no)'] == 1]
print("Представителей классов:")
print("1:", df_1.shape[0])
print("2:", df_2.shape[0])
print("3:", df_3.shape[0])

Сейчас датасет содержит данные о 135 пациентах и обучается на данных о 108 пациентах.
Представителей классов:
1: 61
2: 48
3: 26


In [17]:
def modelInfo(model, train_X, train_Y, features_list, intercept, verbose=False):
    beta0 = 0 if not intercept else model.beta[0]
    beta = model.beta[1:] if intercept else model.beta
    e = (train_Y).astype(np.float32) - beta0
    for i in range(train_X[features_list].shape[1]):
        e -= beta[i] * train_X[features_list[i]].values
    n = train_X[features_list].shape[0]
    k = train_X[features_list].shape[1]
    RSS = np.sum(e ** 2)
    RSE = np.sqrt(RSS / (n - k))
    X = np.hstack((np.ones((n, 1)), train_X[features_list]))
    B = RSE ** 2 * np.linalg.inv(X.T @ X)
    se = []
    for i in range(k + 1):
        se.append(np.sqrt(B[i, i]))
    def pvalue(t_score):
        cdf = ss.norm.cdf(t_score)
        return 2 * min(cdf, 1 - cdf)
    beta = np.insert(beta, 0, beta0)
    head = ('Name', 'Coefficient', 'Std. error', 'norm_statistic', 'p_value')
    if verbose:
        print('{:<42}|{:^15s}|{:^15s}|{:^15s}|{:^15s}|'.format(*head))
    pval = 0.0
    max_col = ''
    intercept_ = intercept
    for i in range(k + 1):
        if i != 0:
            col = train_X[features_list].columns[i - 1]
        else:
            col = 'Intercept'
        if pval < pvalue(beta[i] / se[i]) and i > 0:
            pval = max(pvalue(beta[i] / se[i]), pval)
            max_col = col
        if pvalue(beta[i] / se[i]) > 0.05 and i == 0 and intercept:
            intercept_ = False
        b0_row =(col, beta[i], se[i], beta[i] / se[i], pvalue(beta[i] / se[i]))
        if verbose:
            print('{:<42}|{:^15f}|{:^15f}|{:^15f}|{:^15f}|'.format(*b0_row))
    return pval, max_col, intercept_

In [18]:
diseases_name = [ 'Patient age quantile', 
       'Patient gender', 
       'Hematocrit', 'Hemoglobin', 'Platelets.1',
       'Red blood Cells',
       'Lymphocytes', 'Leukocytes', 'Basophils',
       'Eosinophils #',
       'Basophils #', 'Monocytes #',
       'Red blood cell distribution width (RDW)',
       'Hb saturation (venous blood gas analysis)',
       'Base excess (venous blood gas analysis)',
       'pO2 (venous blood gas analysis)',
       'pH (venous blood gas analysis)',
       'HCO3 (venous blood gas analysis)',  'Neutrophils', 'Urea', 'Proteina C reativa mg/dL',
       'Creatinine', 'Potassium', 'Sodium']
def repeat(X, Y, num, feature_list, intercept):
    frac = int(fraction * X.shape[0])

    train_X, test_X = X[:frac].copy(), X[frac:].copy()
    train_Y, test_Y = Y[:frac].copy(), Y[frac:].copy()
    for i in train_X.index.values:
        for col in train_X.columns:
            if pd.isna(train_X.loc[i][col]):
                train_X.loc[i, col] = train_X[col].mean()

    for i in test_X.index.values:
        for col in test_X.columns:
            if pd.isna(test_X.loc[i][col]):
                test_X.loc[i, col] = test_X[col].mean()
    preds = []
    models = []
    for i in range(3):
        lr = LogisticRegression(fit_intercept=intercept)
        lr.fit(train_X[feature_list], train_Y.values[:, i])
        models.append(lr)
    model = models[num]
    return modelInfo(model, train_X, train_Y.values[:, num], feature_list, intercept)

def exclude_parameters(num):
    k = len(diseases_name)
    intercept = True
    diseases_name_ = diseases_name.copy()
    for j in range(k):
        X = df[diseases_name_]
        Y = df[symptoms_name]
        pval, max_col, intercept = repeat(X, Y, num, diseases_name_, intercept)
        #print(max_col, pval)
        if pval <= 0.05:
            break
        if pval > 0.05:
            diseases_name_.remove(max_col)
    return diseases_name_, intercept
diseases_name_all = []

def find_params(df):
    diseases_name_all = []
    interceptions = []
    for i in range(3):
        ex, intercept = exclude_parameters(i)
        diseases_name_all.append(ex)
        interceptions.append(intercept)
    return diseases_name_all, interceptions

In [22]:
def train(df):
    frac = int(fraction * df.shape[0])
    X = df[diseases_name].copy()
    Y = df[symptoms_name].copy()
    train_Y, test_Y = Y[:frac].copy(), Y[frac:].copy()
    if sum(test_Y.values[:, 0]) == 0 or sum(test_Y.values[:, 1]) == 0 or sum(test_Y.values[:, 2]) == 0:
        return df, False, diseases_name, [], []

    diseases_name_all, interceptions = find_params(df)
    X = df[diseases_name].copy()
    Y = df[symptoms_name].copy()
    train_X, test_X = X[:frac].copy(), X[frac:].copy()
    train_Y, test_Y = Y[:frac].copy(), Y[frac:].copy()
    for i in train_X.index.values:
        for col in train_X.columns:
            if pd.isna(train_X[col].loc[i]):
                train_X.loc[i, col] = train_X[col].mean()

    for i in test_X.index.values:
        for col in test_X.columns:
            if pd.isna(test_X.loc[i][col]):
                test_X.loc[i, col] = test_X[col].mean()

    preds = []
    models = []
    llfs = []
    ll0s = []
    print("Factors:", diseases_name_all)
    print("Use intercept: ", interceptions)
    for i in range(3):
        lr = LogisticRegression(fit_intercept=interceptions[i])
        lr.fit(train_X[diseases_name_all[i]], train_Y.values[:, i])
        models.append(lr)
        pred = lr.predict(test_X[diseases_name_all[i]].values)
        llf = log_loss(test_Y.values[:, i], pred)
        pred = lr.predict_proba(test_X[diseases_name_all[i]].values)
        preds.append(pred.copy())
        lr = LogisticRegression()
        lr.fit(None, train_Y.values[:, i])
        pred = lr.predict(test_X.shape[0])
        ll0 = log_loss(test_Y.values[:, i], pred)
        print("Model #{}".format(i + 1))
        print("McFadden's pseudo r^2: ", 1 - llf/ll0)
        stat = 2*(-llf + ll0)
        print("Statistics value:", stat)
        crit_value = ss.chi2.ppf(0.05, len(diseases_name_all[i]) + 1)
        print("Chi2 distribution critical value:", crit_value)
        if 1 - llf/ll0 < 0.01 and stat <= crit_value:
            return df, False, diseases_name_all, models, interceptions
        
    print("All coefficients were significant!")
    max_pred = []
    preds = np.array(preds).T
    impute = []
    for i in range(preds.shape[1]):
        impute.append(np.argmax(preds[:, i]))
    test_label = []
    for i in range(preds.shape[1]):
        test_label.append(np.argmax(test_Y.values[i]))
    print("Roc-auc score:", roc_auc_score(test_Y.values, preds, average='micro'))
    return df, True, diseases_name_all, models, interceptions

for i in range(1000): # цикл здесь только потому что разбивка на обучающую и тестовую выборки случайна
    print(i)          # и если классы несбалансированы, то одна из моделей может стать незначимой
    df = read_data()
    df, ans, diseases_list, models, interceptions = train(df)

    if ans:
        break
        

0


C:\Users\lissrbay\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log


Factors: [['Platelets.1', 'Monocytes #', 'Hb saturation (venous blood gas analysis)', 'pH (venous blood gas analysis)', 'Proteina C reativa mg/dL'], ['Patient age quantile', 'Leukocytes', 'Monocytes #', 'pO2 (venous blood gas analysis)', 'Sodium'], ['Patient age quantile', 'Platelets.1', 'Leukocytes', 'Eosinophils #', 'Monocytes #', 'Hb saturation (venous blood gas analysis)', 'HCO3 (venous blood gas analysis)', 'Sodium']]
Use intercept:  [False, False, False]
Model #1
McFadden's pseudo r^2:  0.1999907397132884
Statistics value: 5.116618844451889
Chi2 distribution critical value: 1.6353828943279072
Model #2
McFadden's pseudo r^2:  0.09090277707724204
Statistics value: 2.5582501927866694
Chi2 distribution critical value: 1.6353828943279072
Model #3
McFadden's pseudo r^2:  -1.5433811185738833e-05
Statistics value: -0.00023691775709444585
Chi2 distribution critical value: 3.3251128430668158
1


C:\Users\lissrbay\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log


Factors: [['Leukocytes', 'pO2 (venous blood gas analysis)', 'pH (venous blood gas analysis)', 'Proteina C reativa mg/dL'], ['Patient age quantile', 'Leukocytes', 'Monocytes #', 'pO2 (venous blood gas analysis)', 'Sodium'], ['Patient age quantile', 'Leukocytes', 'Monocytes #', 'Hb saturation (venous blood gas analysis)', 'Neutrophils', 'Sodium']]
Use intercept:  [False, False, False]
Model #1
McFadden's pseudo r^2:  0.5999930547849663
Statistics value: 15.350389598309144
Chi2 distribution critical value: 1.1454762260617697
Model #2
McFadden's pseudo r^2:  0.24999807077360203
Statistics value: 7.6752244138742185
Chi2 distribution critical value: 1.6353828943279072
Model #3
McFadden's pseudo r^2:  -2.3150716778497227e-05
Statistics value: -0.0002961471963693896
Chi2 distribution critical value: 2.167349909298058
2
Factors: [['Monocytes #', 'Hb saturation (venous blood gas analysis)', 'pH (venous blood gas analysis)', 'Proteina C reativa mg/dL'], ['Patient age quantile', 'Leukocytes', 'Mon

In [23]:
_, _, diseases_name_all, models, interceptions = train(df)

Factors: [['Monocytes #', 'Hb saturation (venous blood gas analysis)', 'pH (venous blood gas analysis)', 'Proteina C reativa mg/dL'], ['Patient age quantile', 'Leukocytes', 'Monocytes #', 'pO2 (venous blood gas analysis)', 'Sodium'], ['Patient age quantile', 'Leukocytes', 'Monocytes #', 'Hb saturation (venous blood gas analysis)', 'Sodium']]
Use intercept:  [False, False, False]
Model #1
McFadden's pseudo r^2:  0.44443415523698726
Statistics value: 10.233474606660883
Chi2 distribution critical value: 1.1454762260617697
Model #2
McFadden's pseudo r^2:  0.1818118679863332
Statistics value: 5.116678073891173
Chi2 distribution critical value: 1.6353828943279072
Model #3
McFadden's pseudo r^2:  0.5714285714285714
Statistics value: 10.233711524417979
Chi2 distribution critical value: 1.6353828943279072
All coefficients were significant!
Roc-auc score: 0.8216735253772292


In [24]:
for i in range(3):
    print("Model #{}".format(i + 1))
    frac = int(fraction*df.shape[0])
    X = df[diseases_name].copy()
    Y = df[symptoms_name].copy()
    train_X, test_X = X[:frac].copy(), X[frac:].copy()
    train_Y, test_Y = Y[:frac].copy(), Y[frac:].copy()
    for j in train_X.index.values:
        for col in train_X.columns:
            if pd.isna(train_X[col].loc[j]):
                train_X.loc[j, col] = train_X[col].mean()
    train_Y_ = train_Y.values[:, i]
    _, _, _ = modelInfo(models[i], train_X.astype(np.float32), train_Y_, diseases_name_all[i], interceptions[i], True) 
    print()

Model #1
Name                                      |  Coefficient  |  Std. error   |norm_statistic |    p_value    |
Intercept                                 |   0.000000    |   0.303377    |   0.000000    |   1.000000    |
Monocytes #                               |   -0.964766   |   0.327565    |   -2.945267   |   0.003227    |
Hb saturation (venous blood gas analysis) |   -1.700863   |   0.298895    |   -5.690511   |   0.000000    |
pH (venous blood gas analysis)            |   0.957706    |   0.304627    |   3.143863    |   0.001667    |
Proteina C reativa mg/dL                  |   -1.358452   |   0.214692    |   -6.327445   |   0.000000    |

Model #2
Name                                      |  Coefficient  |  Std. error   |norm_statistic |    p_value    |
Intercept                                 |   0.000000    |   1.080333    |   0.000000    |   1.000000    |
Patient age quantile                      |   -0.212863   |   0.076511    |   -2.782109   |   0.005401    |
Leukocyte

In [25]:
frac = int(0.9*df.shape[0])
df[diseases_name_all[0]][:frac].corr()

,Monocytes #,Hb saturation (venous blood gas analysis),pH (venous blood gas analysis),Proteina C reativa mg/dL
Monocytes #,1.000000,0.087012,0.099085,0.383354
Hb saturation (venous blood gas analysis),0.087012,1.000000,0.361730,0.012214
pH (venous blood gas analysis),0.099085,0.361730,1.000000,0.005487
Proteina C reativa mg/dL,0.383354,0.012214,0.005487,1.000000


In [26]:
df[diseases_name_all[1]][:frac].corr()

,Patient age quantile,Leukocytes,Monocytes #,pO2 (venous blood gas analysis),Sodium
Patient age quantile,1.000000,-0.257349,-0.129528,-0.087344,0.058998
Leukocytes,-0.257349,1.000000,0.594263,0.176526,-0.070605
Monocytes #,-0.129528,0.594263,1.000000,0.012902,0.095245
pO2 (venous blood gas analysis),-0.087344,0.176526,0.012902,1.000000,-0.182135
Sodium,0.058998,-0.070605,0.095245,-0.182135,1.000000


In [27]:
df[diseases_name_all[2]][:frac].corr()

,Patient age quantile,Leukocytes,Monocytes #,Hb saturation (venous blood gas analysis),Sodium
Patient age quantile,1.000000,-0.257349,-0.129528,-0.065481,0.058998
Leukocytes,-0.257349,1.000000,0.594263,0.151985,-0.070605
Monocytes #,-0.129528,0.594263,1.000000,0.087012,0.095245
Hb saturation (venous blood gas analysis),-0.065481,0.151985,0.087012,1.000000,-0.173207
Sodium,0.058998,-0.070605,0.095245,-0.173207,1.000000


In [28]:
Далее неоформленная часть, пока заменена другим кодом.

SyntaxError: invalid syntax (<ipython-input-28-2bcf166e0d97>, line 1)

Будем использовать метод один против всех и обучим по классификатору для каждого класса.

In [ ]:
class OnevsRestClassifier():
    def __init__(self, model, params = dict()):
        self.num_classes = 0
        self.clfs = []
        self.model = model
        self.params = params

    def fit(self, X, Y):
        self.num_classes = Y.shape[1]
        classes = Y.columns
        for i in range(self.num_classes):
            clf = self.model(**self.params)
            clf.fit(X, Y[classes[i]])
            self.clfs.append(clf)
        print('Fitted!')

    def predict(self, X): 
        preds = []
        for i in range(self.num_classes):
            pred = self.clfs[i].predict(X)
            preds.append(pred)
        return np.array(preds).T

    def predict_proba(self, X):
        preds = []
        for i in range(self.num_classes):
            pred = self.clfs[i].predict_proba(X)
            preds.append(pred)
        return np.array(preds).T

    def logloss(self, y_true, y_score):
        return -((1 - y_true) * np.log(1 - y_score) + y_true * np.log(y_score)).mean()

    def get_loss(self, y_true, y_score):
        acc = 0
        for i in range(self.num_classes):
            acc += self.logloss(y_true[:, i],y_score[:, i])
        return acc

num_steps = 20000
lr = 1e-2
params = {'lr': lr, 'num_steps':num_steps}
clf = OnevsRestClassifier(LogisticRegression, params)


In [ ]:
def accuracy_hamming(preds, y):
    acc = 0
    tps = []
    tns = []
    for i in range(y.shape[0]):
        y_ = (y.reset_index(drop=True).loc[i]).to_numpy()
        pred_ = preds[i]
        cl = 0
        cr = 0
        for j in range(len(y_)):
            if y_[j] != pred_[j] and y_[j] == 1:
                cl += 1
            if y_[j] != pred_[j] and pred_[j] == 1:
                cr += 1
        acc += (cr + cl) / y.shape[1]
    return acc/(y.shape[0])

print('Hamming multilabel accuracy:', accuracy_hamming(preds, test_Y)) #чем ближе к 0 тем лучше